In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses

2023-03-06 16:51:18.044772: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import cv2
import os
import numpy as np

In [3]:


def loadImages(path, urls,target ):
  images = []
  labels = []
  #for i in range(len(urls))
  for i in range(len(urls)):
    img_path = path + "/" + urls[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    # if we want to resize the images
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(target)
  images = np.asarray(images)
  return images, labels


covid_path = "./data/COVID-19_Radiography_Dataset/COVID/images"
covidUrl = os.listdir(covid_path)
covidImages, covidTargets = loadImages(covid_path, covidUrl, 1)

normal_path = "./data/COVID-19_Radiography_Dataset/Normal/images"
normal_urls = os.listdir(normal_path)
normalImages, normalTargets = loadImages(normal_path, normal_urls, 0)

covidImages=np.asarray(covidImages)

normalImages=np.asarray(normalImages)

data = np.r_[covidImages, normalImages]

targets = np.r_[covidTargets, normalTargets]


from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.25)


# ALEXNET

In [4]:
model = models.Sequential()

model.add(layers.experimental.preprocessing.Resizing(100,100, interpolation="bilinear", input_shape=x_train.shape[1:]))

model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))

model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))

model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 25, 25, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 25, 25, 96)        0         
                                                                 
 activation (Activation)     (None, 25, 25, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 256)         614656    
                                                        

2023-03-06 16:52:15.958666: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 16:52:16.098259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14786 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


_________________________________________________________________


In [5]:
# Compile the model
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


In [6]:
model.fit(x_train, y_train, epochs=10,
            validation_data=(x_test, y_test))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-03-06 16:52:34.820528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-03-06 16:52:35.064715: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0xcbde250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-06 16:52:35.064761: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-03-06 16:52:35.071449: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-06 16:52:35.209059: I tensorflow/compiler/jit/xla_compi

324/324 [==============================] - 8s 13ms/step - loss: 0.6026 - accuracy: 0.7364 - val_loss: 0.5929 - val_accuracy: 0.7274
Epoch 2/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5756 - accuracy: 0.7417 - val_loss: 0.5867 - val_accuracy: 0.7274
Epoch 3/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5757 - accuracy: 0.7417 - val_loss: 0.5861 - val_accuracy: 0.7274
Epoch 4/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5771 - accuracy: 0.7417 - val_loss: 0.5900 - val_accuracy: 0.7274
Epoch 5/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5752 - accuracy: 0.7417 - val_loss: 0.5874 - val_accuracy: 0.7274
Epoch 6/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5734 - accuracy: 0.7417 - val_loss: 0.5918 - val_accuracy: 0.7274
Epoch 7/10
324/324 [==============================] - 3s 10ms/step - loss: 0.5733 - accuracy: 0.7417 - val_loss: 0.5860 - val_accuracy: 0.7274
Epoch 8/10

In [7]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=2)
print("Accuracy", accuracy)
print("Loss", loss)

108/108 - 1s - loss: 0.5936 - accuracy: 0.7274 - 534ms/epoch - 5ms/step
Accuracy 0.7274044156074524
Loss 0.5935896039009094


In [12]:
# find accuracy, precision, recall, f1 score, jaccard index, kappa score, confusion matrix, ROC curve, AUC score, etc.

# precision
from sklearn.metrics import precision_score
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

precision = precision_score(y_test, y_pred, average='macro')
print('Precision: %f' % precision)

# recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred, average='macro')
print('Recall: %f' % recall)

# f1 score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
print('F1 score: %f' % f1)

# jaccard index
from sklearn.metrics import jaccard_score
jaccard = jaccard_score(y_test, y_pred, average='macro')
print('Jaccard score: %f' % jaccard)

# kappa score
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_test, y_pred)
print('Cohens kappa: %f' % kappa)

# confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

# ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(fpr, tpr, thresholds)

# AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print('ROC AUC=%.3f' % (auc))

108/108 [==============================] - 0s 4ms/step
Precision: 0.363702
Recall: 0.500000
F1 score: 0.421097
Jaccard score: 0.363702
Cohens kappa: 0.000000
[[2511    0]
 [ 941    0]]
[0. 1.] [0. 1.] [1 0]
ROC AUC=0.500


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ORIGINAL MODEL

In [14]:
from tensorflow.keras.models import load_model

In [31]:
model_original = load_model('models/base_model.h5')

In [34]:
model_original.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [35]:
model_original.fit(x_train, y_train,epochs=10,validation_data=(x_test, y_test))


Epoch 1/10
324/324 [==============================] - 6s 11ms/step - loss: 0.0517 - accuracy: 0.9822 - val_loss: 0.0745 - val_accuracy: 0.9757
Epoch 2/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0427 - accuracy: 0.9848 - val_loss: 0.0408 - val_accuracy: 0.9846
Epoch 3/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0297 - accuracy: 0.9903 - val_loss: 0.0420 - val_accuracy: 0.9846
Epoch 4/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0349 - accuracy: 0.9874 - val_loss: 0.0604 - val_accuracy: 0.9771
Epoch 5/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0188 - accuracy: 0.9931 - val_loss: 0.0541 - val_accuracy: 0.9806
Epoch 6/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0251 - accuracy: 0.9902 - val_loss: 0.0563 - val_accuracy: 0.9832
Epoch 7/10
324/324 [==============================] - 3s 8ms/step - loss: 0.0147 - accuracy: 0.9951 - val_loss: 0.0631 - val_accuracy: 0.9832
Epoch

In [36]:
loss, accuracy = model_original.evaluate(x_test, y_test, verbose=2)
print("Accuracy", accuracy)
print("Loss", loss)

108/108 - 0s - loss: 0.0793 - accuracy: 0.9745 - 412ms/epoch - 4ms/step
Accuracy 0.9745075106620789
Loss 0.0792604386806488


In [37]:
y_pred = model_original.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

108/108 [==============================] - 0s 3ms/step


In [38]:
# find accuracy, precision, recall, f1 score, jaccard index, kappa score, confusion matrix, ROC curve, AUC score, etc.

# precision
from sklearn.metrics import precision_score


precision = precision_score(y_test, y_pred, average='macro' , labels=None, pos_label=1, sample_weight=None, zero_division='warn')
print('Precision: %f' % precision)

# recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred, average='macro')
print('Recall: %f' % recall)

# f1 score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
print('F1 score: %f' % f1)

# jaccard index
from sklearn.metrics import jaccard_score
jaccard = jaccard_score(y_test, y_pred, average='macro')
print('Jaccard score: %f' % jaccard)

# kappa score
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_test, y_pred , labels=None, weights=None, sample_weight=None)
print('Cohens kappa: %f' % kappa)

# confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

# ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(fpr, tpr, thresholds)

# AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print('ROC AUC=%.3f' % (auc))

Precision: 0.363702
Recall: 0.500000
F1 score: 0.421097
Jaccard score: 0.363702
Cohens kappa: 0.000000
[[2511    0]
 [ 941    0]]
[0. 1.] [0. 1.] [1 0]
ROC AUC=0.500


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# KNN


In [40]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

In [41]:
# train the model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(x_train_flat, y_train)

KNeighborsClassifier(n_neighbors=15)

In [43]:
# evaluate the model
from sklearn.metrics import accuracy_score
y_pred = knn.predict(x_test_flat)

In [44]:
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.8763035921205099


In [45]:
# find accuracy, precision, recall, f1 score, jaccard index, kappa score, confusion matrix, ROC curve, AUC score, etc.

# precision
from sklearn.metrics import precision_score


precision = precision_score(y_test, y_pred, average='macro' , labels=None, pos_label=1, sample_weight=None, zero_division='warn')
print('Precision: %f' % precision)

# recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred, average='macro')
print('Recall: %f' % recall)

# f1 score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
print('F1 score: %f' % f1)

# jaccard index
from sklearn.metrics import jaccard_score
jaccard = jaccard_score(y_test, y_pred, average='macro')
print('Jaccard score: %f' % jaccard)

# kappa score
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_test, y_pred , labels=None, weights=None, sample_weight=None)
print('Cohens kappa: %f' % kappa)

# confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

# ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(fpr, tpr, thresholds)

# AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print('ROC AUC=%.3f' % (auc))

Precision: 0.857081
Recall: 0.820622
F1 score: 0.836017
Jaccard score: 0.726658
Cohens kappa: 0.672759
[[2368  143]
 [ 284  657]]
[0.         0.05694942 1.        ] [0.         0.69819341 1.        ] [2 1 0]
ROC AUC=0.821


In [49]:
# find class distribution in test set
from collections import Counter
counter = Counter(y_test)
print(counter)


Counter({0: 2511, 1: 941})
